In [1]:
# some google colab stuff 
from google.colab import drive

drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [16]:
import codecs
from tqdm import tqdm 
import random 

with codecs.open('/content/drive/My Drive/colab/hin.txt', 'r', encoding='utf-8') as f:
    lines = f.read().split('\n')

print(lines[:2])

['Wow!\tवाह!\tCC-BY 2.0 (France) Attribution: tatoeba.org #52027 (Zifre) & #6179147 (fastrizwaan)', 'Help!\tबचाओ!\tCC-BY 2.0 (France) Attribution: tatoeba.org #435084 (lukaszpp) & #459377 (minshirui)']


In [2]:
input_texts = []
target_texts = []
input_words = []
target_words = []

with codecs.open('/content/drive/My Drive/colab/hin.txt', 'r', encoding='utf-8') as f:
    lines = f.read().split('\n')
    #random.shuffle(lines)
    
    #for line in tqdm(lines[:min(10e13,len(lines) - 1)][:40000]):
    for line in tqdm(lines):
        if len(line.split('\t')) >= 2:
            # As input and target texts are seperated by tab 
            input_text, target_text = line.split('\t')[:2]
            # Adding starting of sequence as '\t' and ending of sequence as '\n'
            target_text = '\t ' + target_text + ' \n'
            
            input_texts.append(input_text)
            target_texts.append(target_text)
            
            # creating list of words for input and target sentences
            for word in input_text.split(" "):
                if word not in input_words:
                    input_words.append(word)
                    
            for word in target_text.split(" "):
                if word not in target_words:
                    target_words.append(word)

100%|██████████| 2779/2779 [00:00<00:00, 7799.01it/s]


In [0]:
target_words = sorted(target_words)

In [0]:
input_words = sorted(input_words)

In [0]:
num_encoder_words = len(input_words)
num_decoder_words = len(target_words)


In [8]:
print(num_encoder_words)
print(num_decoder_words)

3458
3117


In [0]:
# maximum length of encoder and decoder inputs 
max_encoder_seq_len = max([len(i.split(' ')) for i in input_texts])
max_decoder_seq_len = max([len(i.split(' ')) for i in target_texts])

In [10]:
print(max_encoder_seq_len)
print(max_decoder_seq_len)

22
27


In [0]:
# dictionary for index of words to transform text into vectors representing index 
input_word_index = { word: i for i,word in enumerate(input_words)}
reverse_input_word_index = { i:word for i,word in enumerate(input_words)}
target_word_index = { word: i for i,word in enumerate(target_words)}
reverse_target_word_index = { i:word for i,word in enumerate(target_words)}

In [0]:
# Creating empty vectors for representing texts
import numpy as np
encoder_input_data = np.zeros((len(input_texts),max_encoder_seq_len),dtype = 'float32')
decoder_input_data = np.zeros((len(input_texts),max_decoder_seq_len),dtype = 'float32')
decoder_target_data = np.zeros((len(input_texts),max_decoder_seq_len,1),dtype = 'float32')

In [13]:
# Assigning index of word in the empty vectors created above 
for i, (input_text,target_text) in tqdm(enumerate(zip(input_texts,target_texts))):
    for t, word in enumerate(input_text.split(" ")):
        try:
            encoder_input_data[i][t] = input_word_index[word]
        except:
            encoder_input_data[i][t] = num_encoder_words
            
    for t, word in enumerate(target_text.split(" ")):
        try:
            decoder_input_data[i][t] = target_word_index[word]
        except:
            decoder_input_data[i][t] = num_decoder_words
        # as for decoder target data we want to neglect starting word as it will be predicted by encoder genereated context vector    
        if t > 0:
            try:
                decoder_target_data[i][t - 1] = target_word_index[word]
            except:
                 decoder_target_data[i][t - 1] = num_decoder_words
                    
eng_embedding = np.zeros((num_encoder_words + 1,128))
fre_embedding = np.zeros((num_decoder_words + 1,128))

2778it [00:00, 62291.31it/s]


In [14]:
import keras 
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Embedding
from keras.layers import LSTM, BatchNormalization,concatenate,Flatten,Embedding,Dense,Dropout,Reshape,SpatialDropout1D
from keras import Model,Input


Using TensorFlow backend.


In [17]:
embedding_dims = 128
latent_dims = 128
#encoder model
encoder_inp = Input(shape = (None,), name = 'encoder_inp')
encoder_inp_1 = Embedding(num_encoder_words+1, embedding_dims,weights = [eng_embedding])(encoder_inp)
encoder = LSTM(latent_dims, return_state= True, name = 'encoder')
encoder_out, state_h, state_c = encoder(encoder_inp_1)
encoder_states = [state_h, state_c]

#decoder model
decoder_inp = Input(shape = (None,), name = 'decoder_inp')
decoder_inp_1 = Embedding(num_decoder_words+1, embedding_dims, weights = [fre_embedding])(decoder_inp)
decoder_lstm = LSTM(latent_dims, return_state = True, return_sequences= True, name = 'decoder_lstm')
decoder_out, _ , _ = decoder_lstm(decoder_inp_1, initial_state = encoder_states)
decoder_dense = Dense(num_decoder_words, activation='softmax', name = 'decoder_dense')
decoder_out = decoder_dense(decoder_out)

#combined encoder decoder model 
model = Model([encoder_inp, decoder_inp], decoder_out)

model.compile(optimizer='rmsprop',loss='sparse_categorical_crossentropy')

model.fit([encoder_input_data, decoder_input_data],decoder_target_data,batch_size=32,epochs=20,validation_split=0.2)

#model.save('eng_2_french_dumm.h5')

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 2222 samples, validate on 556 samples
Epoch 1/20
2222/2222 [==============================] - 8s 4ms/step - loss: 2.4615 - val_loss: 2.8711
Epoch 2/20
2222/2222 [==============================] - 6s 3ms/step - loss: 1.5288 - val_loss: 2.5975
Epoch 3/20
2222/2222 [==============================] - 6s 3ms/step - loss: 1.4187 - val_loss: 2.6185
Epoch 4/20
2222/2222 [==============================] - 6s 3ms/step - loss: 1.3644 - val_loss: 2.5735
Epoch 5/20
2222/2222 [==============================] - 6s 3ms/step - loss: 1.3279 - val_loss: 2.5847
Epoch 6/20
2222/2222 [==============================] - 6s 3ms/step - loss: 1.2967 - val_loss: 2.5847
Epoch 7/20
2222/2222 [==============================] - 6s 3ms/step - loss: 1.2685 - val_loss: 2.5930
Epoch 8/20
2222/2222 [==============================] - 6s 3ms/step - loss: 1.2411 - val_loss: 2.6070
Epoch 9/20
2222/2222 [==============================] - 6s 3ms/step - loss: 1.2185 - val_loss: 2.5907
Epoch 10/20
2222/2222 [============

In [18]:
# Building encoder/decoder models for prediction
#encoder model 
encoder_model = Model(encoder_inp,encoder_states)

#decoder model 

decoder_inp_h = Input((latent_dims,))
decoder_inp_c = Input((latent_dims,))
decoder_inp_states = [decoder_inp_h, decoder_inp_c]
decoder_out, decoder_out_h, decoder_out_c = decoder_lstm(decoder_inp_1, initial_state = decoder_inp_states)
decoder_out = decoder_dense(decoder_out)
decoder_out_states = [decoder_out_h, decoder_out_c]
decoder_model = Model(inputs = [decoder_inp] + decoder_inp_states, output =[decoder_out]+ decoder_out_states )

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:13: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=[<tf.Tenso..., outputs=[<tf.Tenso...)`
  del sys.path[0]


In [0]:
# model.save('/content/drive/My Drive/colab/eng_2_french_dumm.h5')
# encoder_model.save('/content/drive/My Drive/colab/eng_2_french_encoder_model.h5')
# decoder_model.save('/content/drive/My Drive/colab/eng_2_french_decoder_model.h5')

In [0]:
# creating a testing function to predict output given english sentence
def decode_sequence(input_sentence,encoder_model,decoder_model):
    encoder_input_data = np.zeros((len(input_sentence),max_encoder_seq_len),dtype = 'float32')
    # converting the english sentence to model required vector using word indexes
    for i, input_text in tqdm(enumerate(input_sentence)):
        for t, word in enumerate(input_text.split(" ")):
            try:
                encoder_input_data[i][t] = input_word_index[word]
            except:
                encoder_input_data[i][t] = num_encoder_words
    # calculating the context vector for the first word prediciton            
    states_value = encoder_model.predict(encoder_input_data)
    
    target_seq = np.zeros((1,1))
    # As for the first time model will predict using context vector and '\t' [Start of Sequence]
    target_seq[0][0] = target_word_index['\t']
    stop_condition = False
    decoded_sentence = ''
    
    while not stop_condition:
        output_word,h, c = decoder_model.predict([target_seq] + states_value)
        
        sampled_word_index = np.argmax(output_word[0, -1, :])
        try:
            sampled_char = reverse_target_word_index[sampled_word_index]
        except:
            sampled_char = '<unknown>'
        
        decoded_sentence = decoded_sentence + ' ' + sampled_char
        
        if (sampled_char == '\n' or len(decoded_sentence) > max_decoder_seq_len):
                stop_condition = True
                
        target_seq = np.zeros((1, 1))
        target_seq[0, 0] = sampled_word_index

        states_value = [h, c]
    decoded_sentence = decoded_sentence.replace(' \n','').replace('\n','').strip()
    return decoded_sentence


In [22]:
decode_sequence(["I Love You."],encoder_model,decoder_model)

1it [00:00, 2570.04it/s]


'मैं बहुत प्यार है।'

In [0]:
# Model not working even on train data, need to tune some parameters or use some advanced architectures 